In [16]:
#!/.conda/envs/dp python
# -*- coding: utf-8 -*-

"""
图像转为8位uint格式
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""
from pathlib import Path
import gdal
import os
import ogr
import osr
import sys
import math
from osgeo.ogr import Geometry, Layer
from tqdm import tqdm
import numpy as np
import fnmatch
import copy

In [17]:
def write_img(out_path, im_proj, im_geotrans, im_data):
    """output img

    Args:
        out_path: Output path
        im_proj: Affine transformation parameters
        im_geotrans: spatial reference
        im_data: Output image data

    """
    # identify data type 
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    # calculate number of bands
    if len(im_data.shape) > 2:  
        im_bands, im_height, im_width = im_data.shape
    else:  
        im_bands, (im_height, im_width) = 1, im_data.shape

    # create new img
    driver = gdal.GetDriverByName("GTiff")
    new_dataset = driver.Create(
        out_path, im_width, im_height, im_bands, datatype)
    new_dataset.SetGeoTransform(im_geotrans)
    new_dataset.SetProjection(im_proj)
    if im_bands == 1:
        new_dataset.GetRasterBand(1).WriteArray(im_data)
    else:
        for i in range(im_bands):
            new_dataset.GetRasterBand(i + 1).WriteArray(im_data[i])

    del new_dataset

def read_img(sr_img):
    """read img

    Args:
        sr_img: The full path of the original image

    """
    im_dataset = gdal.Open(sr_img)
    if im_dataset == None:
        print('open sr_img false')
        sys.exit(1)
    im_geotrans = im_dataset.GetGeoTransform()
    im_proj = im_dataset.GetProjection()
    im_width = im_dataset.RasterXSize
    im_height = im_dataset.RasterYSize
    im_data = im_dataset.ReadAsArray(0, 0, im_width, im_height)
    del im_dataset

    return im_data, im_proj, im_geotrans

In [18]:

input_folder = r'E:\project_populus_GF2_and_UAV\0-clip_polygon_img\UAV-321-band\2'  # 输入文件夹路径
output_folder = r'E:\project_populus_GF2_and_UAV\0-clip_polygon_img\UAV-321-8bit'  # 输出文件夹路径

img_type = '*.tif'

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

listpic = fnmatch.filter(os.listdir(input_folder), img_type)

'''逐个读取影像'''
for img in listpic:
    img_full_path = input_folder + '/' + img
    data, proj_temp, geotrans_temp = read_img(img_full_path)
    img_shape = data.shape
    if len(img_shape) == 2:
        output_data = np.zeros((img_shape[0], img_shape[1]), dtype=np.uint8)
        min_value = np.min(data)
        max_value = np.max(data)
        print(min_value, max_value) # 注意最小值和最大值
        output_data = ((data - min_value) / (max_value - min_value) * 255).astype(np.uint8)
    else:
        output_data = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype=np.uint8)
        for i in range(img_shape[0]): # 读取每个波段
            data_temp = data[i, :, :]
            # 对像素值进行缩放和截断操作
            data_temp = np.nan_to_num(data_temp, nan=0) # 处理nan
            min_value = np.min(data_temp)
            max_value = np.max(data_temp)
            print(min_value, max_value) # 注意最小值和最大值
            output_array = ((data_temp - min_value) / (max_value - min_value) * 255).astype(np.uint8)
            output_data[i, :, :] = output_array
    
    output_full_path = output_folder + '/' + img
    write_img(output_full_path, proj_temp, geotrans_temp, output_data)

[[ 570.5743   594.04974  619.5582  ...  313.7253   314.71857  317.2747 ]
 [ 546.1832   579.5193   598.96954 ...  308.0304   313.2539   322.1203 ]
 [ 534.4429   559.51764  581.052   ...  314.0684   313.76047  320.008  ]
 ...
 [1374.4979  1363.4739  1386.6233  ...        nan        nan        nan]
 [1378.4037  1355.3488  1365.0173  ...        nan        nan        nan]
 [   0.         0.         0.      ...    0.         0.         0.     ]]
nan nan
[[ 438.7815   450.2644   457.5575  ...  338.09567  342.1656   334.201  ]
 [ 426.79266  428.78296  421.5588  ...  331.95984  334.2726   328.14212]
 [ 395.11694  393.67822  380.4895  ...  333.3984   336.65826  331.8454 ]
 ...
 [1292.2345  1322.1641  1350.4562  ...        nan        nan        nan]
 [1270.9196  1302.4585  1296.9154  ...        nan        nan        nan]
 [   0.         0.         0.      ...    0.         0.         0.     ]]
nan nan
[[ 382.80463  405.61847  401.42828 ...  293.25293  305.52792  319.20425]
 [ 373.94006  385.0597 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_734364\3401800485.py:32: RuntimeWarning: invalid value encountered in cast
  output_array = ((data_temp - min_value) / (max_value - min_value) * 255).astype(np.uint8)


[[   0.      1761.0181  1751.5024  ...        nan        nan        nan]
 [   0.      1800.9077  1785.0574  ...        nan        nan        nan]
 [   0.      1819.3969  1798.4584  ...        nan        nan        nan]
 ...
 [   0.      1099.5328  1097.4883  ...  710.86346  713.72363  746.32544]
 [   0.      1064.8795  1036.794   ...  680.01886  680.51324  711.65344]
 [   0.       988.2743   952.50366 ...  621.161    616.0575   652.61444]]
nan nan
[[   0.      1684.2454  1682.351   ...        nan        nan        nan]
 [   0.      1661.9554  1670.9175  ...        nan        nan        nan]
 [   0.      1650.163   1664.8197  ...        nan        nan        nan]
 ...
 [   0.       637.2052   627.7434  ...  681.0514   690.18427  705.66125]
 [   0.       666.183    636.16125 ...  652.7274   672.29865  710.0864 ]
 [   0.       681.68884  625.6325  ...  637.95264  663.8001   704.9049 ]]
nan nan
[[   0.      1329.8676  1289.2865  ...        nan        nan        nan]
 [   0.      1331.9143 